# Zoom grader

Loads zoom reports from a pre-defined folder, and calculates attendance.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [ ]:
hide_names = 0
# Set this global var to 1 before pushing to git (preserves anonymity)

In [ ]:
folder_name = '../../data/attendance/'

In [ ]:
# Read all attendance data

df = pd.DataFrame()

file_list = os.listdir(folder_name)
print("Total files: ",len(file_list))

for fname in file_list:
    # print(fname)
    if fname[-3:]!='csv':
        continue # Ignore everything that is not a zoom log
    fullname = folder_name + fname
    header = pd.read_csv(fullname, header=0, nrows=1)
    name = header['Topic'][0]
    datestring = header['Start Time'][0][:10]
    # print(fname, header['Start Time'].values)    
    
    data = pd.read_csv(fullname, header=2)
    data['Date'] = datestring
    data['Meeting'] = name
    data['User Email'] = data['User Email'].fillna('none') # NaNs are ignored by aggregation below
    
    df = df.append(data, ignore_index=True)

In [ ]:
df= df.rename({'User Email': 'email', 
               'Total Duration (Minutes)': 'minutes', 
               'Name (Original Name)': 'name',
               'Meeting':'meeting',
               'Date':'date'}, axis=1)
meeting_dict = {'Neuro Lab - Sep 10': 'Neuro', 
                "Arseny Khakhalin's Zoom Meeting": 'Neuro', 
                "Intro Neuro": 'Neuro',
                'Biosem 00': 'Biosem', 'Bard Biosem': 'Biosem'}
df['meeting'] = df['meeting'].replace(meeting_dict)
df.name = df.name.str.title() # Capitalize (for consistency)
df['check'] = 1*(df.minutes>30)

In [ ]:
# Read the list of students
people = pd.read_csv(folder_name + 'people.txt', header=0, sep='\t')
people['name'] = people['first'] + ' ' + people['last']
people = people.drop_duplicates()
people.columns

In [ ]:
# A list of meetings

dfm = df.groupby(['date','meeting','name']).agg({'minutes': sum})
dfm.minutes = 1*(dfm.minutes>10)
dfm = dfm.groupby(['date', 'meeting']).agg({'minutes': sum}).reset_index()
dfm = dfm.rename({'minutes':'people'}, axis=1)
dfm.date = pd.to_datetime(dfm.date)

plt.figure(figsize=(9,2))
plt.plot(dfm.date.dt.dayofyear, dfm.people, '.');
plt.xlabel('Meeting');
plt.ylabel('People');

In [ ]:
# Looking at attendance of selected students
# df.loc[df.name.str[:3]=='Ser'].sort_values(by='date')

In [ ]:
# ALl possible dates
# sorted(df.date.unique().tolist())

In [ ]:
# Looking at all participants on a selected date
# df.loc[df.date=='10/26/2020'].sort_values(by='name')

In [ ]:
# Iteratively build summaries

dfs = df.groupby(['date', 'meeting', 'name']).agg({'minutes' : sum})
dfs = dfs.reset_index()                                 # Get rid of hierarchical indices
dfs['check'] = 1*(dfs.minutes>30)                       # Participation threshold

dfs = dfs.groupby(['name','meeting']).agg({'check': sum})
dfs = dfs.reset_index()
dfs.name = dfs.name.str.strip()                         # Remove leading and ending spaces

In [ ]:
with pd.option_context('display.max_rows', 1400, 'display.width', 1000):
    if not hide_names:
        #print(dfs)
        pass

In [ ]:
# Use a manually created list of synonims to unify spelling
# Then look for official emails

alts = pd.read_csv(folder_name + 'alt-people.txt', header=0, sep=',')
excluded = pd.read_csv(folder_name + 'exclude.txt', header=0, sep=',')

recognized = dfs.copy().merge(alts, on='name', how='left')
#recognized.name = recognized.name.str.replace('\t',' ')

ind = recognized.translation.notnull()
recognized.loc[ind, 'name'] = recognized.loc[ind,'translation']

recognized = (recognized
              .drop(columns='translation')                            
              .groupby(['name','meeting']).agg({'check': sum}).reset_index()
              .merge(people, on='name', how='left', suffixes=['', '_r'])
              .drop(columns=['first', 'last'])
              )

lost = recognized[recognized.email.isnull()]
lost = (lost.merge(excluded, on='name', how='left', indicator=True).
        query('_merge=="left_only"').
        drop(columns=['_merge','email']))

with pd.option_context('display.max_rows', 1400, 'display.width', 1000):
    if not hide_names:        
        print(lost)

In [ ]:
print('\n'.join(lost.name.values.tolist()))

In [ ]:
# Use a manually created list of synonims to unify spelling

lost = dfs.querya

attempt = lost.merge(alts, on='name', how='inner') # Adds column 'translation'
attempt3 = attempt3[attempt3.translation.notnull()]
attempt3 = attempt3.merge(people[['name','email']], left_on='translation', right_on='name', suffixes=[None, '_y'])
attempt3 = attempt3[['name', 'email']].drop_duplicates()
lost = lost.merge(attempt3['name'], on='name', how='outer', indicator=True).query('_merge=="left_only"')
lost = lost.drop(columns='_merge')

# Any records still unidentified?
lost

In [ ]:
# Identify those who don't have emails set, but have a reasonable name

lost = dfs.query('email=="none"').copy()
lost['name2'] = lost.name.map(lambda s: ' '.join(s.split(' ')[:2])) # Only leave 2 first words
lost.name2 = lost.name2.str.title()                                 # Capitalize every word
lost.name = lost.name.str.title()           # Capitalize first letter of every word
lost = lost[['name', 'name2', 'check']]

# Archetypical filtering operations here (3 times in a row):
# 1. Lookup emails
# 2. Remove those that were found. To avoid massive renaming and dropping columns,
# only look up one column (the name itself).

attempt1 = lost.merge(people[['name', 'email']], on='name', how='inner') # Those with correct name
lost = lost.merge(attempt1['name'], on='name', how='outer', indicator=True, suffixes=[None, '_y']).query('_merge=="left_only"')
lost = lost.drop(columns='_merge')
attempt1 = attempt1[['name', 'email']].drop_duplicates()

# Those with almost correct name (try to interpret first 2 words as a name)
attempt2 = lost.merge(people[['name', 'email']], left_on='name2', right_on='name', how='inner')
# Cannot clean it up yet, still need name2 to project back
lost = lost.merge(attempt2['name2'], on='name2', how='outer', indicator=True).query('_merge=="left_only"')
lost = lost[['name', 'check']]
attempt2 = attempt2.rename({'name_x': 'name'}, axis=1)[['name', 'email']].drop_duplicates()

# Now refer to a manual translation table
alts = pd.read_csv(folder_name + 'alt-people.txt', header=0, sep=',')

attempt3 = lost.merge(alts, on='name', how='inner') # Adds column 'translation'
attempt3 = attempt3[attempt3.translation.notnull()]
attempt3 = attempt3.merge(people[['name','email']], left_on='translation', right_on='name', suffixes=[None, '_y'])
attempt3 = attempt3[['name', 'email']].drop_duplicates()
lost = lost.merge(attempt3['name'], on='name', how='outer', indicator=True).query('_merge=="left_only"')
lost = lost.drop(columns='_merge')

# Any records still unidentified?
lost

In [ ]:
# people.name.values

In [ ]:
# Meetings unidentified people attended
# print(df[['name','date','minutes','meeting']].merge(lost['name'], on='name'))

In [ ]:
# Time to fix emails in the original dataset

df2 = df.copy()
df2.name = df2.name.str.title()             # Standardize capitalization (or aliases table won't work)
df2.loc[df2.email=='none', 'email'] = None
for df_right in [attempt1, attempt2, attempt3]:    
    df2 = df2.merge(df_right, on='name', how='left', suffixes=[None, '_new'])    
    df2.loc[df2.email.isnull(), 'email'] = df2.loc[df2.email.isnull(), 'email_new']
    df2 = df2.drop(columns='email_new')

In [ ]:
# df2.loc[df2.name.str[:3]=='Win'].sort_values(by='date')

In [ ]:
# Summarize
df2 = df2.groupby(['meeting','email']).agg({'check': sum}).reset_index()
df2 = df2.merge(people[['email', 'name']], on='email').reset_index()

df2 = df2[['meeting', 'name', 'email', 'check']]

In [ ]:
def obfuscate(ser):
    out = ser.copy()
    for i in range(len(out)):
        out[i] = 'x'*len(out[i])
    return out

In [ ]:
with pd.option_context('display.max_rows', 14000, 'display.width', 1000):
    df2temp = df2.copy()
    if hide_names:
        df2temp.name = obfuscate(df2temp.name)
        df2temp.email = obfuscate(df2temp.email)
    print(df2temp)
    pass

In [ ]:
# print('\n'.join(df2.loc[df2.meeting=="Neuro"].sort_values(by='name').name.tolist()))

In [ ]:
# df2.loc[df2.meeting=='Neuro'].sort_values(by='name').drop(columns=['meeting', 'email'])